In [1]:
# Data download and unzipping
!wget https://raw.githubusercontent.com/AakashSudhakar/2018-data-science-bowl/master/compressed_files/stage1_test.zip -c
!wget https://raw.githubusercontent.com/AakashSudhakar/2018-data-science-bowl/master/compressed_files/stage1_train.zip -c
# !wget https://raw.githubusercontent.com/ZengChen94/Kaggle-Dataset/master/stage1_test.zip -c
# !wget https://raw.githubusercontent.com/ZengChen94/Kaggle-Dataset/master/stage1_train.zip -c
!mkdir stage1_train stage1_test
!unzip -q stage1_train.zip -d stage1_train/
!unzip -q stage1_test.zip -d stage1_test/

# Downloading and intstalling keras
!pip install keras
!pip install tqdm

--2018-04-03 15:18:17--  https://raw.githubusercontent.com/AakashSudhakar/2018-data-science-bowl/master/compressed_files/stage1_test.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9545388 (9.1M) [application/zip]
Saving to: ‘stage1_test.zip’

stage1_test.zip     100%[===================>]   9.10M  17.6MB/s    in 0.5s    

2018-04-03 15:18:18 (17.6 MB/s) - ‘stage1_test.zip’ saved [9545388/9545388]

--2018-04-03 15:18:19--  https://raw.githubusercontent.com/AakashSudhakar/2018-data-science-bowl/master/compressed_files/stage1_train.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP requ

In [2]:
# Downloading the library Mask_RCNN
!git clone https://github.com/matterport/Mask_RCNN.git
# !git clone https://github.com/ZengChen94/Mask_RCNN.git
!mv ./Mask_RCNN/* ./
!rm -r ./Mask_RCNN
!ls

Cloning into 'Mask_RCNN'...
remote: Counting objects: 369, done.
remote: Total 369 (delta 0), reused 0 (delta 0), pack-reused 369
Receiving objects: 100% (369/369), 50.61 MiB | 26.77 MiB/s, done.
Resolving deltas: 100% (208/208), done.
assets	    inspect_data.ipynb	   README.md	    stage1_train.zip
coco.py     inspect_model.ipynb    samples	    train_shapes.ipynb
config.py   inspect_weights.ipynb  shapes.py	    utils.py
datalab     LICENSE		   stage1_test	    visualize.py
demo.ipynb  model.py		   stage1_test.zip
images	    parallel_model.py	   stage1_train


In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib inline 

In [4]:
import os
import sys
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
import utils
import cv2
from glob import glob
import skimage.io
import seaborn as sns
import sys
import math
import re
import time
import matplotlib
import matplotlib.patches as patches
from config import Config
import model as modellib
import visualize
from model import log

ROOT_DIR = os.getcwd()
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)
TRAIN_PATH = './stage1_train/'
TEST_PATH = './stage1_test/'
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]
kernel = np.ones((1,1),np.uint8)

Using TensorFlow backend.


... done downloading pretrained model!


In [0]:
class KaggleTrainDataset(utils.Dataset):
    def get_path(self, id_):
        return './stage1_train/' + id_ + '/images/' + id_ + '.png'
    
    def load_data(self, train_ids):
        self.add_class("nuclei", 1, "nuclei")
        self.sizes = []
        self.img_ids = train_ids
        for i, id_ in tqdm(enumerate(self.img_ids), total=len(self.img_ids)):
            path = self.get_path(id_)
            img = imread(path)[:, :, :3]
            img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
            w, h = img.shape[0], img.shape[1]
            self.sizes.append((w, h))
            self.add_image("nuclei", image_id = i, path=id_, width=w, height=h)

    def load_image(self, image_id):
        image_name = self.img_ids[image_id]
        path = self.get_path(image_name)
        img = imread(path)[:,:,:3]
        w, h = img.shape[0], img.shape[1]
        return img
    
    def load_mask(self, image_id):
        id_ = self.img_ids[image_id]
        mask_path = './stage1_train/' + id_ + '/masks/'
        num_masks = len(next(os.walk(mask_path))[2])
        width, height = self.sizes[image_id]
        mask = np.zeros((width, height, num_masks), dtype=np.bool)
        for i, mask_file in enumerate(next(os.walk(mask_path))[2]):
            mask_ = imread(mask_path + mask_file)
            mask_ = mask_.reshape((mask_.shape[0], mask_.shape[1], 1))
            mask[:,:,i:i+1] = mask_
        return mask, np.ones(num_masks, np.int32)
      
class KaggleTestDataset(utils.Dataset):
    def get_path(self, id_):
        return './stage1_test/' + id_ + '/images/' + id_ + '.png'
    
    def load_data(self, test_ids):
        self.add_class("nuclei", 1, "nuclei")
        self.sizes = []
        self.img_ids = test_ids
        for i, id_ in tqdm(enumerate(self.img_ids), total=len(self.img_ids)):
            path = self.get_path(id_)
            img = imread(path)[:, :, :3]
            img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
            w, h = img.shape[0], img.shape[1]
            self.sizes.append((w, h))
            self.add_image("nuclei", image_id = i, path=id_, width=w, height=h)

    def load_image(self, image_id):
        image_name = self.img_ids[image_id]
        path = self.get_path(image_name)
        img = imread(path)[:,:,:3]
        w, h = img.shape[0], img.shape[1]
        return img

In [6]:
# Load dataset
dataset_train = KaggleTrainDataset()
dataset_train.load_data(train_ids[:600])
dataset_train.prepare()

dataset_val = KaggleTrainDataset()
# dataset_val.load_data(train_ids[600:670])
dataset_val.load_data(train_ids[600:664])
dataset_val.prepare()

dataset_test = KaggleTestDataset()
dataset_test.load_data(test_ids)
dataset_test.prepare()

100%|██████████| 65/65 [00:00<00:00, 131.77it/s]


In [7]:
# Configurations
class KaggleConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    NAME = "shapes"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 8
    NUM_CLASSES = 1 + 1
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  
    TRAIN_ROIS_PER_IMAGE = 32
    STEPS_PER_EPOCH = 100
    VALIDATION_STEPS = 5
config = KaggleConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_SHAPES                [[128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]
 [  8   8]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     8
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 8
IMAGE_MAX_DIM                  512
IMAGE_MIN_DIM                  512
IMAGE_PADDING                  True
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9]
MINI_MASK_SHAPE                (56, 56)
NAME                           shapes
NUM_CLASSES                    2
POOL_SIZE              

In [0]:
# Notebook Preferences
def get_ax(rows=1, cols=1, size=8):
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [0]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

# Weights to start with
init_with = "coco"  # imagenet, coco, or last
if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last()[1], by_name=True)

In [0]:
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

# # Train the head branches
# # Passing layers="heads" freezes all layers except the head
# # layers. You can also pass a regular expression to select
# # which layers to train by name pattern.
# model.train(dataset_train, dataset_val, 
#             learning_rate=config.LEARNING_RATE, 
#             epochs=1, layers='heads')

In [0]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

!pip install imgaug
import imgaug
augmentation = imgaug.augmenters.Fliplr(0.5)
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)
model_path = model.find_last()[1]
model.load_weights(model_path, by_name=True)

# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=70, 
            layers="all",
            augmentation=augmentation)

[autoreload of httplib2 failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ValueError: __init__() requires a code object with 0 free vars, not 1
]



Starting at epoch 51. LR=0.0001

Checkpoint Path: /content/logs/shapes20180403T1519/mask_rcnn_shapes_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branch2b         (Conv2D)
bn3a_branch2b        

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:2087: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 52/70
  9/100 [=>............................] - ETA: 13:25 - loss: 0.8633 - rpn_class_loss: 0.0365 - rpn_bbox_loss: 0.3351 - mrcnn_class_loss: 0.1371 - mrcnn_bbox_loss: 0.1448 - mrcnn_mask_loss: 0.2098

In [0]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes.h5")
model.keras_model.save_weights(model_path)

In [13]:
# Detection
class InferenceConfig(KaggleConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()[1]

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

Loading weights from  /content/logs/shapes20180403T1519/mask_rcnn_shapes_0050.h5


In [0]:
# # Test on a random image
# image_id = random.choice(dataset_val.image_ids)

# original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
#     modellib.load_image_gt(dataset_val, inference_config, 
#                            image_id, use_mini_mask=False)

# # Ground truth
# log("original_image", original_image)
# log("image_meta", image_meta)
# log("gt_class_id", gt_class_id)
# log("gt_bbox", gt_bbox)
# log("gt_mask", gt_mask)

# visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
#                             dataset_train.class_names, figsize=(8, 8))

In [0]:
# results = model.detect([original_image], verbose=1)

# r = results[0]
# visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
#                             dataset_val.class_names, r['scores'], ax=get_ax())

# # r['masks'].shape = 256*256*num
# # plt.imshow(r['masks'][:,:,0])

In [0]:
# # Compute VOC-Style mAP @ IoU=0.5
# # Running on 10 images. Increase for better accuracy.
# image_ids = np.random.choice(dataset_val.image_ids, 10)
# APs = []
# for image_id in image_ids:
#     # Load image and ground truth data
#     image, image_meta, gt_class_id, gt_bbox, gt_mask =\
#         modellib.load_image_gt(dataset_val, inference_config,
#                                image_id, use_mini_mask=False)
#     molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
#     # Run object detection
#     results = model.detect([image], verbose=0)
#     r = results[0]
#     # Compute AP
#     AP, precisions, recalls, overlaps =\
#         utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
#                          r["rois"], r["class_ids"], r["scores"], r['masks'])
#     APs.append(AP)
    
# print("mAP: ", np.mean(APs))

In [0]:
# # rle method-1
# # Run-length encoding stolen from https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
# def rle_encoding(x):
#     dots = np.where(x.T.flatten() == 1)[0]
#     run_lengths = []
#     prev = -2
#     for b in dots:
#         if (b>prev+1): run_lengths.extend((b + 1, 0))
#         run_lengths[-1] += 1
#         prev = b
#     return run_lengths

# def prob_to_rles(x, cutoff=0.5):
#     lab_img = label(x > cutoff)
#     for i in range(1, lab_img.max() + 1):
#         yield rle_encoding(lab_img == i)
        
# # Iterate over the test IDs and generate run-length encodings for each seperate mask identified by skimage
# def mask_to_rle(img_id, mask, test_ids, rles):
#     rle = list(prob_to_rles(mask))
#     rles.extend(rle)
#     test_ids.extend([img_id] * len(rle))
#     return test_ids,rles

In [0]:
# test_ids = []
# rles = []

# for image_id in dataset_test.image_ids:
#     img_id = dataset_test.img_ids[image_id]
#     image = dataset_test.load_image(image_id)
#     molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
#     # Run object detection
#     results = model.detect([image], verbose=0)
#     r = results[0]
#     mask = r['masks'].max(axis=2)
#     test_ids, rles = mask_to_rle(img_id, mask, test_ids, rles)

# saved_file = 'rcnn-1.csv'    

# print (len(list(set(test_ids))))

# # Create submission DataFrame
# sub = pd.DataFrame()
# sub['ImageId'] = test_ids
# sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
# sub.to_csv(saved_file, index=False)

# # Install the PyDrive wrapper & import libraries.
# # This only needs to be done once in a notebook.
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # Authenticate and create the PyDrive client.
# # This only needs to be done once in a notebook.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# # Create & upload a file.
# uploaded = drive.CreateFile({'title': saved_file})
# uploaded.SetContentFile(saved_file)
# uploaded.Upload()
# print('Uploaded file with ID {}'.format(uploaded.get('id')))
# # 4/AACG6-C0U8VukJ2_YnpV2e6QZ59968sOM1sdmryFRHG8oWfN3oS9DDE

In [0]:
# rle method-2
def numpy2encoding_no_overlap2(predicts, img_id):
    sum_predicts = np.sum(predicts, axis=2)
    rows, cols = np.where(sum_predicts>=2)
    
    for i in zip(rows, cols):
        instance_indicies = np.where(np.any(predicts[i[0],i[1],:]))[0]
        highest = instance_indicies[0]
        predicts[i[0],i[1],:] = predicts[i[0],i[1],:]*0
        predicts[i[0],i[1],highest] = 1
    
    ImageId = []
    EncodedPixels = []
    for i in range(predicts.shape[2]): 
        rle = run_length_encoding(predicts[:,:,i])
        if len(rle)>0:
            ImageId.append(img_id)
            EncodedPixels.append(rle)    
    return ImageId, EncodedPixels

def run_length_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    run_lengths = ' '.join([str(r) for r in run_lengths])
    return run_lengths

In [0]:
ImageId = []
EncodedPixels = []

for image_id in dataset_test.image_ids:
    img_id = dataset_test.img_ids[image_id]
    image = dataset_test.load_image(image_id)
    results = model.detect([image], verbose=0)
    r = results[0]
#     idx = utils.non_max_suppression(r['rois'], r['scores'], 0.2)
#     ImageId_batch, EncodedPixels_batch = numpy2encoding_no_overlap2(r['masks'][idx], img_id)
    ImageId_batch, EncodedPixels_batch = numpy2encoding_no_overlap2(r['masks'], img_id)
    ImageId += ImageId_batch
    EncodedPixels += EncodedPixels_batch
    
saved_file = 'rcnn-6.csv'

# Create submission DataFrame
sub = pd.DataFrame()
sub['ImageId'] = ImageId
# sub['EncodedPixels'] = pd.Series(EncodedPixels).apply(lambda x: ''.join(str(y) for y in x))
sub['EncodedPixels'] = EncodedPixels
sub.to_csv(saved_file, index=False)

# # Install the PyDrive wrapper & import libraries.
# # This only needs to be done once in a notebook.
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # Authenticate and create the PyDrive client.
# # This only needs to be done once in a notebook.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# # Create & upload a file.
# uploaded = drive.CreateFile({'title': saved_file})
# uploaded.SetContentFile(saved_file)
# uploaded.Upload()
# print('Uploaded file with ID {}'.format(uploaded.get('id')))

from google.colab import files

files.download(saved_file)

In [23]:
# Save the model to Google Drive
!ls

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

saved_file = 'mask_rcnn_coco.h5'

# Create & upload a file.
uploaded = drive.CreateFile({'title': saved_file})
uploaded.SetContentFile(saved_file)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

# saved_file = 'mask_rcnn_coco.h5'
# files.download(saved_file)

Uploaded file with ID 1OXCGxu3Gzcv6zWsIpdi22hJZ1DUg3Ibz
